# Testing models

----


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-config" data-toc-modified-id="Set-config-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set config</a></span></li><li><span><a href="#Collect-cases" data-toc-modified-id="Collect-cases-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Collect cases</a></span></li><li><span><a href="#Treat-databases" data-toc-modified-id="Treat-databases-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Treat databases</a></span></li><li><span><a href="#Run-model" data-toc-modified-id="Run-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run model</a></span><ul class="toc-item"><li><span><a href="#Testing-plot" data-toc-modified-id="Testing-plot-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Testing plot</a></span></li></ul></li><li><span><a href="#Compare-hospital-capacity" data-toc-modified-id="Compare-hospital-capacity-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compare hospital capacity</a></span></li><li><span><a href="#Draft" data-toc-modified-id="Draft-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Draft</a></span></li></ul></div>

## Set config

In [96]:
import pandas as pd
import numpy as np
import plotly.express as px
import yaml
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.dates as md

from paths import *
from scripts import get_data, clean_data

%reload_ext autoreload
%autoreload 2

config = yaml.load(open('../configs/config.yaml', 'r'), Loader=yaml.FullLoader)

from datetime import datetime
date_time = datetime.today().strftime('%Y-%m-%d-%H-%M')

## Collect cases

In [6]:
config['get_data_paths']

{'cases_covid19br': 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv',
 'cases_brasilio': 'https://brasil.io/api/dataset/covid19/caso/data?format=json'}

In [7]:
df_cases_covid19br = get_data.city_cases_covid19br(config['get_data_paths']['cases_covid19br'])
df_cases_brasilio = get_data.city_cases_brasilio(config['get_data_paths']['cases_brasilio'])

In [10]:
df_cases_covid19br.head()

,date,country,state,city,newCases,totalCases,last_update
583,2020-03-20,Brazil,TOTAL,TOTAL,318,1012,2020-03-23 14:43:46.846335
506,2020-03-20,Brazil,SC,Chapecó/SC,1,1,2020-03-23 14:43:46.846335
492,2020-03-20,Brazil,RS,Torres/RS,1,2,2020-03-23 14:43:46.846335
493,2020-03-20,Brazil,BA,Lauro de Freitas/BA,1,3,2020-03-23 14:43:46.846335
494,2020-03-20,Brazil,RS,Alvorada/RS,1,2,2020-03-23 14:43:46.846335


In [11]:
df_cases_brasilio.head()

,city,city_ibge_code,confirmed,confirmed_per_100k_inhabitants,date,death_rate,deaths,estimated_population_2019,is_last,place_type,state,last_update
0,Igaracy,2607.0,1,16.34788,2020-03-23,NaN,0,6117.0,True,city,PB,2020-03-23 14:43:48.358659
1,João Pessoa,7507.0,1,0.12361,2020-03-23,NaN,0,809015.0,True,city,PB,2020-03-23 14:43:48.358659
2,None,25.0,2,0.04977,2020-03-23,NaN,0,4018127.0,True,state,PB,2020-03-23 14:43:48.358659
3,Rio Branco,401.0,11,2.70059,2020-03-22,NaN,0,407319.0,True,city,AC,2020-03-23 14:43:48.358659
4,None,12.0,11,1.24726,2020-03-22,NaN,0,881935.0,True,state,AC,2020-03-23 14:43:48.358659


## Treat databases

In [12]:
config['raw_paths'], config['treated_paths']

({'cases_covid19br': 'city_cases_covid19br.csv',
  'cases_brasilio': 'city_cases_brasilio.csv',
  'sus': 'covid19_SUS_database.csv'},
 {'cases_covid19br': 'last_cases_covid19br.csv',
  'cases_brasilio': 'last_cases_covid19br.csv',
  'sus': 'treated_covid19_SUS_database.csv'})

In [13]:
df_cases_covid19br = clean_data.treat_covid19br(config['raw_paths']['cases_covid19br'], config['treated_paths']['cases_covid19br'])
df_cases_brasilio = clean_data.treat_brasilio(config['raw_paths']['cases_brasilio'], config['treated_paths']['cases_brasilio']) 

In [14]:
df_cases_covid19br.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122 entries, 34 to 122
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   region_id   122 non-null    object
 1   city        122 non-null    object
 2   place_type  122 non-null    object
 3   date        122 non-null    object
 4   confirmed   122 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 5.7+ KB


In [15]:
df_cases_brasilio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147 entries, 383 to 480
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   city                            147 non-null    object 
 1   city_ibge_code                  146 non-null    float64
 2   confirmed                       147 non-null    int64  
 3   confirmed_per_100k_inhabitants  146 non-null    float64
 4   date                            147 non-null    object 
 5   death_rate                      4 non-null      float64
 6   deaths                          147 non-null    int64  
 7   estimated_population_2019       146 non-null    float64
 8   is_last                         147 non-null    bool   
 9   place_type                      147 non-null    object 
 10  state                           147 non-null    object 
 11  last_update                     147 non-null    object 
 12  region_id                       14

In [16]:
print('Total cases covid19br:', df_cases_covid19br['confirmed'].sum())
print('Total cases brasilio:', df_cases_brasilio[df_cases_brasilio['place_type'] != 'state']['confirmed'].sum())

Total cases covid19br: 1012
Total cases brasilio: 1274


In [152]:
df_sus = clean_data.treat_sus(config['raw_paths']['sus'], config['treated_paths']['sus'])
df_sus

,region_id,municipio,uf,populacao,quantidade_leitos,ventiladores_existentes
0,AC,AC,AC,881935,1484.0,152.0
1,AL,AL,AL,3337357,5891.0,550.0
2,AM,AM,AM,4144597,5700.0,899.0
3,AP,AP,AP,845731,1098.0,94.0
4,BA,BA,BA,14873064,28960.0,3194.0
...,...,...,...,...,...,...
5565,VIANOPOLIS GO,Vianópolis,GO,13863,40.0,1.0
5566,VICENTINOPOLIS GO,Vicentinópolis,GO,8743,16.0,1.0
5567,VILA BOA GO,Vila Boa,GO,6171,13.0,0.0
5568,VILA PROPICIO GO,Vila Propício,GO,5821,0.0,0.0


## Run model

Running SIR model with different level of output.

From [Modeling COVID-19 Spread vs Healthcare Capacity
](https://alhill.shinyapps.io/COVID19seir/):

>* Mild infection (I1) - These individuals have symptoms like fever and cough and may have mild pneumonia. **Hospitalization is not required** (though in many countries such individuals are also hospitalized)

>* Severe infection (I2) - These individuals have more severe pneumonia that leads to dyspnea, respiratory frequency <30/min, blood oxygen saturation <93%, partial pressure of arterial oxygen to fraction of inspired oxygen ratio <300, and/or lung infiltrates >50% within 24 to 48 hours. **Hospitalization and supplemental oxygen are generally required.**

>* Critical infection (I3) - These individuals experience respiratory failure, septic shock, and/or multiple organ dysfunction or failure. **Treatment in an ICU, often with mechanical ventilation, is required.**


In [18]:
# merge sus and cases data
df = pd.merge(df_cases_brasilio, df_sus, how='left', on='region_id')
df

,city,city_ibge_code,confirmed,confirmed_per_100k_inhabitants,date,death_rate,deaths,estimated_population_2019,is_last,place_type,state,last_update,region_id,municipio,uf,populacao,quantidade_leitos,ventiladores_disponiveis
0,São Paulo,50308.0,358,2.92197,2020-03-20,0.0251,9,12252023.0,True,city,SP,2020-03-23 14:43:48.358659,SAO PAULO SP,São Paulo,SP,12252023.0,27847.0,276.0
1,Rio de Janeiro,4557.0,168,2.50041,2020-03-22,NaN,0,6718903.0,True,city,RJ,2020-03-23 14:43:48.358659,RIO DE JANEIRO RJ,Rio de Janeiro,RJ,6718903.0,13933.0,377.0
2,Brasília,108.0,117,3.88025,2020-03-22,NaN,0,3015268.0,True,city,DF,2020-03-23 14:43:48.358659,BRASILIA DF,Brasília,DF,3015268.0,6705.0,112.0
3,Fortaleza,4400.0,116,4.34564,2020-03-22,NaN,0,2669342.0,True,city,CE,2020-03-23 14:43:48.358659,FORTALEZA CE,Fortaleza,CE,2669342.0,8287.0,40.0
4,Porto Alegre,14902.0,38,2.56104,2020-03-22,NaN,0,1483771.0,True,city,RS,2020-03-23 14:43:48.358659,PORTO ALEGRE RS,Porto Alegre,RS,1483771.0,7019.0,154.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Ji-Paraná,122.0,1,0.77538,2020-03-22,NaN,0,128969.0,True,city,RO,2020-03-23 14:43:48.358659,JI-PARANA RO,Ji-Paraná,RO,128969.0,259.0,0.0
143,Miguel Pereira,2908.0,1,3.91573,2020-03-22,1.0000,1,25538.0,True,city,RJ,2020-03-23 14:43:48.358659,MIGUEL PEREIRA RJ,Miguel Pereira,RJ,25538.0,99.0,0.0
144,Guapimirim,1850.0,1,1.65243,2020-03-22,NaN,0,60517.0,True,city,RJ,2020-03-23 14:43:48.358659,GUAPIMIRIM RJ,Guapimirim,RJ,60517.0,137.0,0.0
145,Barra Mansa,407.0,1,0.54226,2020-03-22,NaN,0,184412.0,True,city,RJ,2020-03-23 14:43:48.358659,BARRA MANSA RJ,Barra Mansa,RJ,184412.0,296.0,1.0


In [39]:
from models import sir

In [131]:
df_final = pd.DataFrame()

for region_id in df['region_id'].unique():
    for scenario, s_values  in config['scenarios'].items():
        for bound, r0 in s_values['R0'].items():
            
            df_city = df.query(f"region_id == '{region_id}'")

            current_state = {
                'population': df_city['populacao'].values[0],
                'current_infected': df_city['confirmed'].values[0]
            }

            config['model_parameters']['sir']['R0'] = r0

            res = sir.entrypoint(current_state, config['model_parameters']['sir'])
            
            res['region_id'] = region_id
            res['scenario'] = scenario
            res['bound'] = bound
            
            df_final = pd.concat([df_final, res], axis=0)

# df_final.to_csv(OUTPUT_PATH / 'model_for_city_{}.csv'.format(study), index=False)
# df_final.to_csv('../data/output/model_for_city.csv', index=False)

In [132]:
df_final['municipio'] = df_final['region_id'].apply(lambda x: x[:-2])
df_final['uf'] = df_final['region_id'].apply(lambda x: x[-2:])

# cols = ['region_id','quantidade_leitos','ventiladores_disponiveis','populacao']
# df_final = df_final.merge(df_sus[cols],on='region_id')

# mask = df_cases_brasilio['is_last']==True
# df_final = df_final.merge(df_cases_brasilio[mask][['region_id','confirmed','date']], on='region_id')

In [133]:
# df_final.to_csv(OUTPUT_PATH / 'model_for_city.csv', index=False)
df_final.to_parquet(OUTPUT_PATH / 'model_for_city.parquet', index=False)

### Testing plot

In [134]:
# sp = df_final[df_final['region_id'] == 'SAO PAULO SP']
# sp['days'] = sp.index
# sp.head()

In [135]:
# fig = px.line(sp, x='days', y='I', color='scenario')
# fig

## Compare hospital capacity

In [153]:
args_dday = {'critical': {'infected': 'I3', 'resource': 'ventiladores_existentes'}, 
             'severe': {'infected': 'I2', 'resource': 'quantidade_leitos'}}

In [195]:
def calculate_dday(df_sus, df_final, args, perc):
    
    cols = ['region_id', 'scenario', 'days'] + [args['infected'], args['resource']]
    
    df = df_sus.merge(df_final, on='region_id')[cols]
    
    
    df = df[df[args['resource']]  *  perc / 100 < df[args['infected']]] # days with overload
    dday = df.sort_values(['region_id', 'days']).drop_duplicates(subset = ['region_id','scenario'], keep='first') # dday: first overload day
    dday = dday.rename({'days': 'dday_{}'.format(args['infected'])}, axis=1)
    dday['available_{}'.format(args['resource'])] = df[args['resource']]  *  perc / 100
    dday['perc_{}'.format(args['resource'])] = perc
#     dday['municipio']= dday_critical['region_id'].apply(lambda x: x[:-2])
#     dday['uf'] = dday_critical['region_id'].apply(lambda x: x[-2:])
    
    return dday

In [191]:
dday_critical = pd.DataFrame()
for perc in range(1,101):
    
    df_aux = calculate_dday(df_sus,df_final ,args_dday['critical'], perc)
    dday_critical = pd.concat([dday_critical,df_aux] , axis=0)

dday_critical.head()

,region_id,scenario,dday_I3,I3,ventiladores_existentes,available_ventiladores_existentes,perc_ventiladores_existentes
241560,ALVORADA RS,nothing,0.0,0.100000,9.0,0.09,1
242292,ALVORADA RS,isolation,0.0,0.100000,9.0,0.09,1
243024,ALVORADA RS,lockdown,0.0,0.100000,9.0,0.09,1
305627,ANAPOLIS GO,nothing,17.0,1.210018,112.0,1.12,1
306368,ANAPOLIS GO,isolation,26.0,1.131948,112.0,1.12,1


In [192]:
dday_severe = pd.DataFrame()
for perc in range(1,101):
    
    df_aux = calculate_dday(df_sus,df_final ,args_dday['severe'], perc)
    dday_severe = pd.concat([dday_severe,df_aux] , axis=0)

dday_severe.head()

,region_id,scenario,dday_I2,I2,quantidade_leitos,available_quantidade_leitos,perc_quantidade_leitos
241935,ALVORADA RS,nothing,9.0,1.122974,97.0,0.97,1
242671,ALVORADA RS,isolation,13.0,1.009352,97.0,0.97,1
243426,ALVORADA RS,lockdown,36.0,0.995906,97.0,0.97,1
305634,ANAPOLIS GO,nothing,24.0,10.131818,1012.0,10.12,1
306380,ANAPOLIS GO,isolation,38.0,10.404148,1012.0,10.12,1


In [196]:
dday_critical.to_parquet(OUTPUT_PATH / 'dday_critical.parquet', index=False)
dday_critical.to_csv(OUTPUT_PATH / 'dday_critical.csv', index=False)

dday_severe.to_parquet(OUTPUT_PATH / 'dday_severe.parquet', index=False)
dday_critical.to_csv(OUTPUT_PATH / 'dday_critical.csv', index=False)


In [55]:
t = dday[dday['municipio'] != dday['uf']]
px.histogram(t, 'days', title='Número de cidades por qtd de dias para colapso hospitalar (sem intervenção)')

KeyError: 'municipio'

In [156]:
t.min()

region_id            ALVORADA RS
municipio               Alvorada
uf                            AC
populacao                   2878
quantidade_leitos              0
scenario                 nothing
days                           1
I3                    0.00350567
dtype: object

In [157]:
len(df_capacity['region_id'].unique()) # All cities/states reach max capacity doing nothing

142

## Draft

In [28]:
# def model(current_state, model_paramenters):
    
#     current_state = {
#         'population'
#         'current_infected'
#     }
    
#     model_parameters = {
#         'R0'
#         'sick_days'
#         'i2_percentage'
#         'i3_percentage'
#     }
    
#     return {
#         'day'
#         'S'
#         'R'
#         'I1'
#         'I2'
#         'I3'
#     }

In [ ]:
# # UTI demand
# uti_demand = I * perc_infectados_internacao / 100

# # Create output dataframe
# dd = pd.DataFrame([municipio]*days,columns=['municipio'])

# dd['uf']=df_city['uf'].values[0]
# dd['place_type'] = df_city['place_type'].values[0]
# dd['populacao'] = N
# dd['dias_doente'] = dias_doente
# dd['percentual_infectados_que_precisam_de_internacao'] = perc_infectados_internacao
# dd['data_numero_infectados'] = df_city['date'].values[0]
# dd['numero_infectados'] = I_i0

# n_beds = df_city['quantidade_leitos'].values[0]
# dd['leitos_total'] = n_beds

# dd['dias'] = t
# dd['demanda_por_uti'] = uti_demand
# dd['cenario'] = scenario
# dd['suscetiveis'] = S
# dd['infectados'] = I
# dd['recuperados'] = R


# return dd

In [138]:
# municipio = 'São Paulo'
# mask = df_final['municipio']== municipio
# dd = df_final[mask]

# perc = 80
# fig = px.line(dd, x='dias',y='demanda_por_uti', color='cenario')
# fig

In [12]:
# df_final = pd.DataFrame()

# for study in parameters_statics.keys():
#     dic_study = parameters_statics[study]
    
#     for municipio in df['municipio'].unique():
#         for scenario in dic_study.keys():

#             mask = df['municipio'] == municipio
#             df_city = df[mask]
            
#             dd = sir_results(df_city, scenario, dias_doente, perc_infectados_internacao, study)
#             df_final = pd.concat([df_final,dd],axis=0)

#     df_final.to_csv(OUTPUT_PATH / 'model_for_city_{}.csv'.format(study), index=False)
        
# # df_final.to_csv('../data/output/model_for_city.csv', index=False)

- MODELO (antigo)

  - SIR - https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/
  - https://colab.research.google.com/drive/1bmR-W2NvQ7gPnldTfei927NtGJUCC1BB#scrollTo=47Er1T5YZBXf

    ro - fixo para cada cidade
    populacao de cada cidade